Esta notebook contiene bloques de código útiles para realizar Q-learning en el entorno "Pendulum"

In [ ]:
#!pip install gymnasium==0.28.1
#!pip install numpy
#!pip install wandb

import numpy as np
from pendulum_env_extended import PendulumEnvExtended
import random
import gymnasium as gym
import wandb

In [ ]:
# Inicializar wandb
wandb.init(project="pendulum-qlearning", name="pendulum-V1-run")

# Inicializar el entorno
env = PendulumEnvExtended(render_mode='rgb_array')

Discretización de los estados

In [ ]:
x_space = np.linspace(-1, 1, 10)
y_space = np.linspace(-1, 1, 10)
vel_space = np.linspace(-8, 8, 100)
x_space

Obtener el estado a partir de la observación

In [ ]:
def get_state(obs):
    x, y, vel = obs
    x_bin = np.digitize(x, x_space)
    y_bin = np.digitize(y, y_space)
    vel_bin = np.digitize(vel, vel_space)
    return x_bin, y_bin, vel_bin

In [ ]:
state = get_state(np.array([-0.4, 0.2, 0.3]))
state

Discretización de las acciones

In [ ]:
actions = list(np.linspace(-2, 2, 10))
actions

In [ ]:
def get_sample_action():
    return random.choice(actions)

Inicilización de la tabla Q

In [ ]:
Q = np.zeros((len(x_space) + 1, len(y_space) + 1, len(vel_space) + 1, len(actions)))
Q

Obtención de la acción a partir de la tabla Q

In [ ]:
def optimal_policy(state, Q):
    action = actions[np.argmax(Q[state])]
    return action

Epsilon-Greedy Policy

In [ ]:
def epsilon_greedy_policy(state, Q, epsilon=0.1):
    explore = np.random.binomial(1, epsilon)
    if explore:
        action = get_sample_action()
        print('explore')
    # exploit
    else:
        action = optimal_policy(state, Q)
        print('exploit')
        
    return action

Ejemplo de episodio 

In [ ]:
obs,_ = env.reset()
print(obs)
done = False
total_reward = 0
state = get_state(obs)
while not done:
    # Acción del modelo
    action = epsilon_greedy_policy(state, Q, 0.5)
    
    # Indice de la accion en Q
    action_idx = actions.index(action)
    
    # Acción del ambiente
    real_action = np.array([action])
     
    obs, reward, done, _, _ = env.step(real_action)
    next_state = get_state(obs)
    
   # Usar action_idx para actualizar Q
   
    total_reward += reward
    print('->', state, action, reward, obs, done)
    env.render()
print('total_reward', total_reward)